In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import display

import os
import copy
import pickle
import sys
import gc
import time
import json
import importlib

In [4]:
sys.path.insert(0, "../draughts")
import board_proc as bp
import engine
import board

importlib.reload(bp);
importlib.reload(engine);
importlib.reload(board);

**Let's generate data**

In [5]:
b = board.Board()
b

. o . o . o . o
o . o . o . o .
. o . o . o . o
. . . . . . . .
. . . . . . . .
O . O . O . O .
. O . O . O . O
O . O . O . O .

In [6]:
b.engine.turn

True

In [8]:
b.valid_moves()

(a3b4, c3b4, c3d4, e3d4, e3f4, g3f4, g3h4)

In [11]:
white_input_data = []
black_input_data = []

### Generate legal moves.

In [129]:
def gen_legal_moves():
    for i in range(500):
        b = board.Board()
        for j in range(200):
            try:
                valid_moves = b.valid_moves()
                if not valid_moves:
                    break
                ind = np.random.randint(len(valid_moves))
                move = str(valid_moves[ind])
                inp = bp.board_to_input(b._engine.layout)
                for pos in [*bp.board_pos_to_num_pos(move[:2]), *bp.board_pos_to_num_pos(move[2:])]:
                    base = np.zeros(8, dtype=np.int8)
                    base[pos] = 1
#                     print(pos)
                    inp.extend(base)

                if b.engine.turn:
                    white_input_data.append(inp)
                else:
                    black_input_data.append(inp)
#                 print(move)
#                 output_data.append(1)
                b.push_move(valid_moves[ind])
            except Exception as ex:
                print(b)
                raise ex
        print(f"{i} games passed.", end="\r")
            
gen_legal_moves()

In [130]:
np.array(white_input_data[120]).reshape(-1, 8)

array([[0, 0, 0, 0, 0, 3, 0, 0],
       [3, 0, 3, 0, 3, 0, 3, 0],
       [0, 4, 0, 3, 0, 3, 0, 1],
       [0, 0, 0, 0, 3, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 3, 0, 3, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0]])

In [104]:
len(white_input_data[0]), len(black_input_data[0]), 

(96, 96)

In [105]:
len(white_input_data), len(black_input_data)

(37799, 37581)

In [125]:
illegal_white_input = []
illegal_black_input = []

## Generate illegal moves

In [126]:
def get_illegal_move(b):
    x1 = np.random.randint(8)
    y1 = np.random.randint(8)
    x2 = np.random.randint(8)
    y2 = np.random.randint(8)
    rand_move = f"{bp.num_pos_to_board_pos((x1, y1))}{bp.num_pos_to_board_pos((x2, y2))}"
    for m in b.valid_moves():
        if str(m) != rand_move:
            return x1, y1, x2, y2
    return get_illegal_move(b)

def gen_illegal_moves():
    for i in range(1000):
        b = board.Board()
        for j in range(300):
            try:
                valid_moves = b.valid_moves()
                if not valid_moves:
                    break
#                 move = str(valid_moves[ind])
                inp = bp.board_to_input(b._engine.layout)
                for pos in get_illegal_move(b):
                    base = np.zeros(8, dtype=np.int8)
                    base[pos] = 1
                    inp.extend(base)
                if b.engine.turn:
                    illegal_white_input.append(inp)
                else:
                    illegal_black_input.append(inp)
#                 print(move)
                ind = np.random.randint(len(valid_moves))
                b.push_move(valid_moves[ind])
            except Exception as ex:
                print(b)
                raise ex
        print(f"{i} gamse passed.", end="\r")

gen_illegal_moves()

In [127]:
np.array(illegal_white_input[0]).reshape(-1, 8)

array([[0, 3, 0, 3, 0, 3, 0, 3],
       [3, 0, 3, 0, 3, 0, 3, 0],
       [0, 3, 0, 3, 0, 3, 0, 3],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

In [ ]:
len(illegal_white_input[0]), len(illegal_black_input[0])

In [93]:
len(illegal_white_input), len(illegal_black_input)

(79300, 78802)

In [147]:
white_input_data = np.array(white_input_data)
black_input_data = np.array(black_input_data)

illegal_white_input = np.array(illegal_white_input)
illegal_black_input = np.array(illegal_black_input)

white_input_data.shape, black_input_data.shape, illegal_white_input.shape, illegal_black_input.shape

((38395, 96), (38187, 96), (77436, 96), (76936, 96))

In [157]:
w_input_data = np.concatenate((white_input_data, illegal_white_input))
b_input_data = np.concatenate((black_input_data, illegal_black_input))

In [158]:
# generate output binary labels
w_output_data = np.concatenate((np.ones(len(white_input_data)), np.zeros(len(illegal_white_input))))
b_output_data = np.concatenate((np.ones(len(black_input_data)), np.zeros(len(illegal_black_input))))

In [159]:
w_input_data.shape, w_output_data.shape, b_input_data.shape, b_output_data.shape

((115831, 96), (115831,), (115123, 96), (115123,))

In [162]:
w_input_data[0][:-32].reshape(-1, 8), w_input_data[0][-32:].reshape(-1, 8), w_output_data[0]

(array([[0, 3, 0, 3, 0, 3, 0, 3],
        [3, 0, 3, 0, 3, 0, 3, 0],
        [0, 3, 0, 3, 0, 3, 0, 3],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0, 1, 0]]),
 array([[0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0]]),
 1.0)

In [163]:
w_input_data[100][:-32].reshape(-1, 8), w_input_data[100][-32:].reshape(-1, 8), w_output_data[100]

(array([[0, 3, 0, 3, 0, 3, 0, 3],
        [3, 0, 3, 0, 3, 0, 3, 0],
        [0, 3, 0, 3, 0, 3, 0, 3],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0, 1, 0]]),
 array([[0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0]]),
 1.0)

In [165]:
with open("../dataset/checkers_white_input.pickle", "wb") as file:
    pickle.dump(w_input_data, file)
    
with open("../dataset/checkers_white_output.pickle", "wb") as file:
    pickle.dump(w_output_data, file)
        
with open("../dataset/checkers_black_input.pickle", "wb") as file:
    pickle.dump(b_input_data, file)
    
with open("../dataset/checkers_black_output.pickle", "wb") as file:
    pickle.dump(b_output_data, file)

### Leaving this for now.

In [444]:
data, counts = np.unique(input_data, return_counts=True, axis=0)

In [486]:
sorted(counts)[-10:]

[120, 130, 130, 669, 686, 701, 715, 741, 749, 754]

In [489]:
# counts.sort()
indices = np.where(counts > 600)[0]
np.where(counts > 600)[0].shape, indices

((7,), array([2228083, 2228199, 2228201, 2228320, 2228321, 2228438, 2228440]))

In [503]:
exclude_indices = []
for i in indices:
    exclude_indices.extend(np.where(np.all(input_data == data[i], axis=1))[0][1:])

In [506]:
np.arange(input_data.shape[0])

array([      0,       1,       2, ..., 2348307, 2348308, 2348309])

In [513]:
include_indices = []
for i in np.arange(input_data.shape[0]):
    if not i in exclude_indices:
        include_indices.append(i)

In [518]:
len(include_indices), input_data.shape[0]

(2343302, 2348310)

In [517]:
include_indices = np.array(include_indices)

In [519]:
input_data = input_data[include_indices]
output_data = output_data[include_indices]

In [520]:
random_ind = np.random.permutation(len(input_data))
random_ind.max(), random_ind.min(), random_ind

(2343301,
 0,
 array([1217134,  141328,  311597, ..., 1721870, 1687128, 1195036]))

In [521]:
input_data = input_data[random_ind]
output_data = output_data[random_ind]

In [522]:
with open("../dataset/checkers_input.pickle", "wb") as file:
    pickle.dump(input_data, file)
    
with open("../dataset/checkers_output.pickle", "wb") as file:
    pickle.dump(output_data, file)

In [523]:
data, counts = np.unique(input_data, return_counts=True, axis=0)

In [524]:
sorted(counts)[-10:]

[112, 112, 112, 113, 113, 118, 120, 120, 130, 130]

### test gen data

In [4]:
with open("../dataset/checkers_input.pickle", "rb") as file:
    X = pickle.load(file)
    
with open("../dataset/checkers_output.pickle", "rb") as file:
    y = pickle.load(file)

In [60]:
def see_data(i):
    bd = bp.input_to_board(X[i][:-4])
    pos = X[i][-4:]
#     print(pos)
    move = f"{bp.num_pos_to_board_pos(pos[:2])}{bp.num_pos_to_board_pos(pos[2:])}"
    print(np.array(bd).reshape(8, 8))
    print(move)
    print(bool(y[i]))

In [76]:
i = np.where(y == 1)[0]

In [83]:
ind = np.random.randint(len(i))
print(ind)
see_data(i[ind])

130641
[['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' 'x' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' 'X']
 ['.' '.' '.' '.' '.' '.' '.' '.']]
c5e3
True


### Generate legal jump games

In [7]:
input_data = []
output_data = []

In [12]:
def gen_legal_moves():
    for i in range(5000):
        b = board.Board()
        for j in range(300):
            try:
                valid_moves = b.valid_moves()
                if not valid_moves:
                    break
                ind = np.random.randint(len(valid_moves))
                move = str(valid_moves[ind])
                inp = bp.board_to_input(b._engine.layout)
                inp.extend([*bp.board_pos_to_num_pos(move[:2]), *bp.board_pos_to_num_pos(move[2:])])
                input_data.append(inp)
#                 print(move)
                output_data.append(1)
                b.push_move(valid_moves[ind])
            except Exception as ex:
                print(b)
                raise ex
        print(f"{i} game passed.")
            
gen_legal_moves()

0 game passed.
1 game passed.
2 game passed.
3 game passed.
4 game passed.
5 game passed.
6 game passed.
7 game passed.
8 game passed.
9 game passed.
10 game passed.
11 game passed.
12 game passed.
13 game passed.
14 game passed.
15 game passed.
16 game passed.
17 game passed.
18 game passed.
19 game passed.
20 game passed.
21 game passed.
22 game passed.
23 game passed.
24 game passed.
25 game passed.
26 game passed.
27 game passed.
28 game passed.
29 game passed.
30 game passed.
31 game passed.
32 game passed.
33 game passed.
34 game passed.
35 game passed.
36 game passed.
37 game passed.
38 game passed.
39 game passed.
40 game passed.
41 game passed.
42 game passed.
43 game passed.
44 game passed.
45 game passed.
46 game passed.
47 game passed.
48 game passed.
49 game passed.
50 game passed.
51 game passed.
52 game passed.
53 game passed.
54 game passed.
55 game passed.
56 game passed.
57 game passed.
58 game passed.
59 game passed.
60 game passed.
61 game passed.
62 game passed.
63

494 game passed.
495 game passed.
496 game passed.
497 game passed.
498 game passed.
499 game passed.
500 game passed.
501 game passed.
502 game passed.
503 game passed.
504 game passed.
505 game passed.
506 game passed.
507 game passed.
508 game passed.
509 game passed.
510 game passed.
511 game passed.
512 game passed.
513 game passed.
514 game passed.
515 game passed.
516 game passed.
517 game passed.
518 game passed.
519 game passed.
520 game passed.
521 game passed.
522 game passed.
523 game passed.
524 game passed.
525 game passed.
526 game passed.
527 game passed.
528 game passed.
529 game passed.
530 game passed.
531 game passed.
532 game passed.
533 game passed.
534 game passed.
535 game passed.
536 game passed.
537 game passed.
538 game passed.
539 game passed.
540 game passed.
541 game passed.
542 game passed.
543 game passed.
544 game passed.
545 game passed.
546 game passed.
547 game passed.
548 game passed.
549 game passed.
550 game passed.
551 game passed.
552 game passe

979 game passed.
980 game passed.
981 game passed.
982 game passed.
983 game passed.
984 game passed.
985 game passed.
986 game passed.
987 game passed.
988 game passed.
989 game passed.
990 game passed.
991 game passed.
992 game passed.
993 game passed.
994 game passed.
995 game passed.
996 game passed.
997 game passed.
998 game passed.
999 game passed.
1000 game passed.
1001 game passed.
1002 game passed.
1003 game passed.
1004 game passed.
1005 game passed.
1006 game passed.
1007 game passed.
1008 game passed.
1009 game passed.
1010 game passed.
1011 game passed.
1012 game passed.
1013 game passed.
1014 game passed.
1015 game passed.
1016 game passed.
1017 game passed.
1018 game passed.
1019 game passed.
1020 game passed.
1021 game passed.
1022 game passed.
1023 game passed.
1024 game passed.
1025 game passed.
1026 game passed.
1027 game passed.
1028 game passed.
1029 game passed.
1030 game passed.
1031 game passed.
1032 game passed.
1033 game passed.
1034 game passed.
1035 game pas

1441 game passed.
1442 game passed.
1443 game passed.
1444 game passed.
1445 game passed.
1446 game passed.
1447 game passed.
1448 game passed.
1449 game passed.
1450 game passed.
1451 game passed.
1452 game passed.
1453 game passed.
1454 game passed.
1455 game passed.
1456 game passed.
1457 game passed.
1458 game passed.
1459 game passed.
1460 game passed.
1461 game passed.
1462 game passed.
1463 game passed.
1464 game passed.
1465 game passed.
1466 game passed.
1467 game passed.
1468 game passed.
1469 game passed.
1470 game passed.
1471 game passed.
1472 game passed.
1473 game passed.
1474 game passed.
1475 game passed.
1476 game passed.
1477 game passed.
1478 game passed.
1479 game passed.
1480 game passed.
1481 game passed.
1482 game passed.
1483 game passed.
1484 game passed.
1485 game passed.
1486 game passed.
1487 game passed.
1488 game passed.
1489 game passed.
1490 game passed.
1491 game passed.
1492 game passed.
1493 game passed.
1494 game passed.
1495 game passed.
1496 game 

1901 game passed.
1902 game passed.
1903 game passed.
1904 game passed.
1905 game passed.
1906 game passed.
1907 game passed.
1908 game passed.
1909 game passed.
1910 game passed.
1911 game passed.
1912 game passed.
1913 game passed.
1914 game passed.
1915 game passed.
1916 game passed.
1917 game passed.
1918 game passed.
1919 game passed.
1920 game passed.
1921 game passed.
1922 game passed.
1923 game passed.
1924 game passed.
1925 game passed.
1926 game passed.
1927 game passed.
1928 game passed.
1929 game passed.
1930 game passed.
1931 game passed.
1932 game passed.
1933 game passed.
1934 game passed.
1935 game passed.
1936 game passed.
1937 game passed.
1938 game passed.
1939 game passed.
1940 game passed.
1941 game passed.
1942 game passed.
1943 game passed.
1944 game passed.
1945 game passed.
1946 game passed.
1947 game passed.
1948 game passed.
1949 game passed.
1950 game passed.
1951 game passed.
1952 game passed.
1953 game passed.
1954 game passed.
1955 game passed.
1956 game 

...

2823 game passed.
2824 game passed.
2825 game passed.
2826 game passed.
2827 game passed.
2828 game passed.
2829 game passed.
2830 game passed.
2831 game passed.
2832 game passed.
2833 game passed.
2834 game passed.
2835 game passed.
2836 game passed.
2837 game passed.
2838 game passed.
2839 game passed.
2840 game passed.
2841 game passed.
2842 game passed.
2843 game passed.
2844 game passed.
2845 game passed.
2846 game passed.
2847 game passed.
2848 game passed.
2849 game passed.
2850 game passed.
2851 game passed.
2852 game passed.
2853 game passed.
2854 game passed.
2855 game passed.
2856 game passed.
2857 game passed.
2858 game passed.
2859 game passed.
2860 game passed.
2861 game passed.
2862 game passed.
2863 game passed.
2864 game passed.
2865 game passed.
2866 game passed.
2867 game passed.
2868 game passed.
2869 game passed.
2870 game passed.
2871 game passed.
2872 game passed.
2873 game passed.
2874 game passed.
2875 game passed.
2876 game passed.
2877 game passed.
2878 game 

3282 game passed.
3283 game passed.
3284 game passed.
3285 game passed.
3286 game passed.
3287 game passed.
3288 game passed.
3289 game passed.
3290 game passed.
3291 game passed.
3292 game passed.
3293 game passed.
3294 game passed.
3295 game passed.
3296 game passed.
3297 game passed.
3298 game passed.
3299 game passed.
3300 game passed.
3301 game passed.
3302 game passed.
3303 game passed.
3304 game passed.
3305 game passed.
3306 game passed.
3307 game passed.
3308 game passed.
3309 game passed.
3310 game passed.
3311 game passed.
3312 game passed.
3313 game passed.
3314 game passed.
3315 game passed.
3316 game passed.
3317 game passed.
3318 game passed.
3319 game passed.
3320 game passed.
3321 game passed.
3322 game passed.
3323 game passed.
3324 game passed.
3325 game passed.
3326 game passed.
3327 game passed.
3328 game passed.
3329 game passed.
3330 game passed.
3331 game passed.
3332 game passed.
3333 game passed.
3334 game passed.
3335 game passed.
3336 game passed.
3337 game 

3741 game passed.
3742 game passed.
3743 game passed.
3744 game passed.
3745 game passed.
3746 game passed.
3747 game passed.
3748 game passed.
3749 game passed.
3750 game passed.
3751 game passed.
3752 game passed.
3753 game passed.
3754 game passed.
3755 game passed.
3756 game passed.
3757 game passed.
3758 game passed.
3759 game passed.
3760 game passed.
3761 game passed.
3762 game passed.
3763 game passed.
3764 game passed.
3765 game passed.
3766 game passed.
3767 game passed.
3768 game passed.
3769 game passed.
3770 game passed.
3771 game passed.
3772 game passed.
3773 game passed.
3774 game passed.
3775 game passed.
3776 game passed.
3777 game passed.
3778 game passed.
3779 game passed.
3780 game passed.
3781 game passed.
3782 game passed.
3783 game passed.
3784 game passed.
3785 game passed.
3786 game passed.
3787 game passed.
3788 game passed.
3789 game passed.
3790 game passed.
3791 game passed.
3792 game passed.
3793 game passed.
3794 game passed.
3795 game passed.
3796 game 

4197 game passed.
4198 game passed.
4199 game passed.
4200 game passed.
4201 game passed.
4202 game passed.
4203 game passed.
4204 game passed.
4205 game passed.
4206 game passed.
4207 game passed.
4208 game passed.
4209 game passed.
4210 game passed.
4211 game passed.
4212 game passed.
4213 game passed.
4214 game passed.
4215 game passed.
4216 game passed.
4217 game passed.
4218 game passed.
4219 game passed.
4220 game passed.
4221 game passed.
4222 game passed.
4223 game passed.
4224 game passed.
4225 game passed.
4226 game passed.
4227 game passed.
4228 game passed.
4229 game passed.
4230 game passed.
4231 game passed.
4232 game passed.
4233 game passed.
4234 game passed.
4235 game passed.
4236 game passed.
4237 game passed.
4238 game passed.
4239 game passed.
4240 game passed.
4241 game passed.
4242 game passed.
4243 game passed.
4244 game passed.
4245 game passed.
4246 game passed.
4247 game passed.
4248 game passed.
4249 game passed.
4250 game passed.
4251 game passed.
4252 game 

4657 game passed.
4658 game passed.
4659 game passed.
4660 game passed.
4661 game passed.
4662 game passed.
4663 game passed.
4664 game passed.
4665 game passed.
4666 game passed.
4667 game passed.
4668 game passed.
4669 game passed.
4670 game passed.
4671 game passed.
4672 game passed.
4673 game passed.
4674 game passed.
4675 game passed.
4676 game passed.
4677 game passed.
4678 game passed.
4679 game passed.
4680 game passed.
4681 game passed.
4682 game passed.
4683 game passed.
4684 game passed.
4685 game passed.
4686 game passed.
4687 game passed.
4688 game passed.
4689 game passed.
4690 game passed.
4691 game passed.
4692 game passed.
4693 game passed.
4694 game passed.
4695 game passed.
4696 game passed.
4697 game passed.
4698 game passed.
4699 game passed.
4700 game passed.
4701 game passed.
4702 game passed.
4703 game passed.
4704 game passed.
4705 game passed.
4706 game passed.
4707 game passed.
4708 game passed.
4709 game passed.
4710 game passed.
4711 game passed.
4712 game 

In [13]:
len(input_data), len(output_data)

(782979, 782979)

In [14]:
def get_illegal_move(b):
    x1 = np.random.randint(8)
    y1 = np.random.randint(8)
    x2 = np.random.randint(8)
    y2 = np.random.randint(8)
    rand_move = f"{bp.num_pos_to_board_pos((x1, y1))}{bp.num_pos_to_board_pos((x2, y2))}"
    for m in b.valid_moves():
        if str(m) != rand_move:
            return x1, y1, x2, y2
    return get_illegal_move(b)

def gen_illegal_moves():
    for i in range(10000):
        b = board.Board()
        for j in range(300):
            try:
                valid_moves = b.valid_moves()
                if not valid_moves:
                    break
                ind = np.random.randint(len(valid_moves))
#                 move = str(valid_moves[ind])
                inp = bp.board_to_input(b._engine.layout)
                inp.extend(get_illegal_move(b))
                input_data.append(inp)
#                 print(move)
                output_data.append(0)
                b.push_move(valid_moves[ind])
            except Exception as ex:
                print(b)
                raise ex
        print(f"{i} game passed.")
            
gen_illegal_moves()

0 game passed.
1 game passed.
2 game passed.
3 game passed.
4 game passed.
5 game passed.
6 game passed.
7 game passed.
8 game passed.
9 game passed.
10 game passed.
11 game passed.
12 game passed.
13 game passed.
14 game passed.
15 game passed.
16 game passed.
17 game passed.
18 game passed.
19 game passed.
20 game passed.
21 game passed.
22 game passed.
23 game passed.
24 game passed.
25 game passed.
26 game passed.
27 game passed.
28 game passed.
29 game passed.
30 game passed.
31 game passed.
32 game passed.
33 game passed.
34 game passed.
35 game passed.
36 game passed.
37 game passed.
38 game passed.
39 game passed.
40 game passed.
41 game passed.
42 game passed.
43 game passed.
44 game passed.
45 game passed.
46 game passed.
47 game passed.
48 game passed.
49 game passed.
50 game passed.
51 game passed.
52 game passed.
53 game passed.
54 game passed.
55 game passed.
56 game passed.
57 game passed.
58 game passed.
59 game passed.
60 game passed.
61 game passed.
62 game passed.
63

492 game passed.
493 game passed.
494 game passed.
495 game passed.
496 game passed.
497 game passed.
498 game passed.
499 game passed.
500 game passed.
501 game passed.
502 game passed.
503 game passed.
504 game passed.
505 game passed.
506 game passed.
507 game passed.
508 game passed.
509 game passed.
510 game passed.
511 game passed.
512 game passed.
513 game passed.
514 game passed.
515 game passed.
516 game passed.
517 game passed.
518 game passed.
519 game passed.
520 game passed.
521 game passed.
522 game passed.
523 game passed.
524 game passed.
525 game passed.
526 game passed.
527 game passed.
528 game passed.
529 game passed.
530 game passed.
531 game passed.
532 game passed.
533 game passed.
534 game passed.
535 game passed.
536 game passed.
537 game passed.
538 game passed.
539 game passed.
540 game passed.
541 game passed.
542 game passed.
543 game passed.
544 game passed.
545 game passed.
546 game passed.
547 game passed.
548 game passed.
549 game passed.
550 game passe

977 game passed.
978 game passed.
979 game passed.
980 game passed.
981 game passed.
982 game passed.
983 game passed.
984 game passed.
985 game passed.
986 game passed.
987 game passed.
988 game passed.
989 game passed.
990 game passed.
991 game passed.
992 game passed.
993 game passed.
994 game passed.
995 game passed.
996 game passed.
997 game passed.
998 game passed.
999 game passed.
1000 game passed.
1001 game passed.
1002 game passed.
1003 game passed.
1004 game passed.
1005 game passed.
1006 game passed.
1007 game passed.
1008 game passed.
1009 game passed.
1010 game passed.
1011 game passed.
1012 game passed.
1013 game passed.
1014 game passed.
1015 game passed.
1016 game passed.
1017 game passed.
1018 game passed.
1019 game passed.
1020 game passed.
1021 game passed.
1022 game passed.
1023 game passed.
1024 game passed.
1025 game passed.
1026 game passed.
1027 game passed.
1028 game passed.
1029 game passed.
1030 game passed.
1031 game passed.
1032 game passed.
1033 game passe

1437 game passed.
1438 game passed.
1439 game passed.
1440 game passed.
1441 game passed.
1442 game passed.
1443 game passed.
1444 game passed.
1445 game passed.
1446 game passed.
1447 game passed.
1448 game passed.
1449 game passed.
1450 game passed.
1451 game passed.
1452 game passed.
1453 game passed.
1454 game passed.
1455 game passed.
1456 game passed.
1457 game passed.
1458 game passed.
1459 game passed.
1460 game passed.
1461 game passed.
1462 game passed.
1463 game passed.
1464 game passed.
1465 game passed.
1466 game passed.
1467 game passed.
1468 game passed.
1469 game passed.
1470 game passed.
1471 game passed.
1472 game passed.
1473 game passed.
1474 game passed.
1475 game passed.
1476 game passed.
1477 game passed.
1478 game passed.
1479 game passed.
1480 game passed.
1481 game passed.
1482 game passed.
1483 game passed.
1484 game passed.
1485 game passed.
1486 game passed.
1487 game passed.
1488 game passed.
1489 game passed.
1490 game passed.
1491 game passed.
1492 game 

1896 game passed.
1897 game passed.
1898 game passed.
1899 game passed.
1900 game passed.
1901 game passed.
1902 game passed.
1903 game passed.
1904 game passed.
1905 game passed.
1906 game passed.
1907 game passed.
1908 game passed.
1909 game passed.
1910 game passed.
1911 game passed.
1912 game passed.
1913 game passed.
1914 game passed.
1915 game passed.
1916 game passed.
1917 game passed.
1918 game passed.
1919 game passed.
1920 game passed.
1921 game passed.
1922 game passed.
1923 game passed.
1924 game passed.
1925 game passed.
1926 game passed.
1927 game passed.
1928 game passed.
1929 game passed.
1930 game passed.
1931 game passed.
1932 game passed.
1933 game passed.
1934 game passed.
1935 game passed.
1936 game passed.
1937 game passed.
1938 game passed.
1939 game passed.
1940 game passed.
1941 game passed.
1942 game passed.
1943 game passed.
1944 game passed.
1945 game passed.
1946 game passed.
1947 game passed.
1948 game passed.
1949 game passed.
1950 game passed.
1951 game 

...

7847 game passed.
7848 game passed.
7849 game passed.
7850 game passed.
7851 game passed.
7852 game passed.
7853 game passed.
7854 game passed.
7855 game passed.
7856 game passed.
7857 game passed.
7858 game passed.
7859 game passed.
7860 game passed.
7861 game passed.
7862 game passed.
7863 game passed.
7864 game passed.
7865 game passed.
7866 game passed.
7867 game passed.
7868 game passed.
7869 game passed.
7870 game passed.
7871 game passed.
7872 game passed.
7873 game passed.
7874 game passed.
7875 game passed.
7876 game passed.
7877 game passed.
7878 game passed.
7879 game passed.
7880 game passed.
7881 game passed.
7882 game passed.
7883 game passed.
7884 game passed.
7885 game passed.
7886 game passed.
7887 game passed.
7888 game passed.
7889 game passed.
7890 game passed.
7891 game passed.
7892 game passed.
7893 game passed.
7894 game passed.
7895 game passed.
7896 game passed.
7897 game passed.
7898 game passed.
7899 game passed.
7900 game passed.
7901 game passed.
7902 game 

8306 game passed.
8307 game passed.
8308 game passed.
8309 game passed.
8310 game passed.
8311 game passed.
8312 game passed.
8313 game passed.
8314 game passed.
8315 game passed.
8316 game passed.
8317 game passed.
8318 game passed.
8319 game passed.
8320 game passed.
8321 game passed.
8322 game passed.
8323 game passed.
8324 game passed.
8325 game passed.
8326 game passed.
8327 game passed.
8328 game passed.
8329 game passed.
8330 game passed.
8331 game passed.
8332 game passed.
8333 game passed.
8334 game passed.
8335 game passed.
8336 game passed.
8337 game passed.
8338 game passed.
8339 game passed.
8340 game passed.
8341 game passed.
8342 game passed.
8343 game passed.
8344 game passed.
8345 game passed.
8346 game passed.
8347 game passed.
8348 game passed.
8349 game passed.
8350 game passed.
8351 game passed.
8352 game passed.
8353 game passed.
8354 game passed.
8355 game passed.
8356 game passed.
8357 game passed.
8358 game passed.
8359 game passed.
8360 game passed.
8361 game 

8765 game passed.
8766 game passed.
8767 game passed.
8768 game passed.
8769 game passed.
8770 game passed.
8771 game passed.
8772 game passed.
8773 game passed.
8774 game passed.
8775 game passed.
8776 game passed.
8777 game passed.
8778 game passed.
8779 game passed.
8780 game passed.
8781 game passed.
8782 game passed.
8783 game passed.
8784 game passed.
8785 game passed.
8786 game passed.
8787 game passed.
8788 game passed.
8789 game passed.
8790 game passed.
8791 game passed.
8792 game passed.
8793 game passed.
8794 game passed.
8795 game passed.
8796 game passed.
8797 game passed.
8798 game passed.
8799 game passed.
8800 game passed.
8801 game passed.
8802 game passed.
8803 game passed.
8804 game passed.
8805 game passed.
8806 game passed.
8807 game passed.
8808 game passed.
8809 game passed.
8810 game passed.
8811 game passed.
8812 game passed.
8813 game passed.
8814 game passed.
8815 game passed.
8816 game passed.
8817 game passed.
8818 game passed.
8819 game passed.
8820 game 

9223 game passed.
9224 game passed.
9225 game passed.
9226 game passed.
9227 game passed.
9228 game passed.
9229 game passed.
9230 game passed.
9231 game passed.
9232 game passed.
9233 game passed.
9234 game passed.
9235 game passed.
9236 game passed.
9237 game passed.
9238 game passed.
9239 game passed.
9240 game passed.
9241 game passed.
9242 game passed.
9243 game passed.
9244 game passed.
9245 game passed.
9246 game passed.
9247 game passed.
9248 game passed.
9249 game passed.
9250 game passed.
9251 game passed.
9252 game passed.
9253 game passed.
9254 game passed.
9255 game passed.
9256 game passed.
9257 game passed.
9258 game passed.
9259 game passed.
9260 game passed.
9261 game passed.
9262 game passed.
9263 game passed.
9264 game passed.
9265 game passed.
9266 game passed.
9267 game passed.
9268 game passed.
9269 game passed.
9270 game passed.
9271 game passed.
9272 game passed.
9273 game passed.
9274 game passed.
9275 game passed.
9276 game passed.
9277 game passed.
9278 game 

9682 game passed.
9683 game passed.
9684 game passed.
9685 game passed.
9686 game passed.
9687 game passed.
9688 game passed.
9689 game passed.
9690 game passed.
9691 game passed.
9692 game passed.
9693 game passed.
9694 game passed.
9695 game passed.
9696 game passed.
9697 game passed.
9698 game passed.
9699 game passed.
9700 game passed.
9701 game passed.
9702 game passed.
9703 game passed.
9704 game passed.
9705 game passed.
9706 game passed.
9707 game passed.
9708 game passed.
9709 game passed.
9710 game passed.
9711 game passed.
9712 game passed.
9713 game passed.
9714 game passed.
9715 game passed.
9716 game passed.
9717 game passed.
9718 game passed.
9719 game passed.
9720 game passed.
9721 game passed.
9722 game passed.
9723 game passed.
9724 game passed.
9725 game passed.
9726 game passed.
9727 game passed.
9728 game passed.
9729 game passed.
9730 game passed.
9731 game passed.
9732 game passed.
9733 game passed.
9734 game passed.
9735 game passed.
9736 game passed.
9737 game 

In [15]:
input_data = np.array(input_data)
output_data = np.array(output_data)

In [16]:
input_data.shape, output_data.shape

((2337721, 68), (2337721,))

In [17]:
data, counts = np.unique(input_data, return_counts=True, axis=0)

In [18]:
sorted(counts)[-10:]

[123, 126, 129, 678, 705, 718, 718, 719, 724, 751]

In [19]:
indices = np.where(counts > 600)[0]
np.where(counts > 600)[0].shape, indices

((7,), array([2213643, 2213753, 2213755, 2213873, 2213875, 2213993, 2213995]))

In [20]:
exclude_indices = []
for i in indices:
    exclude_indices.extend(np.where(np.all(input_data == data[i], axis=1))[0][1:])

In [21]:
include_indices = []
for i in np.arange(input_data.shape[0]):
    if not i in exclude_indices:
        include_indices.append(i)

In [22]:
len(include_indices), input_data.shape[0]

(2332715, 2337721)

In [23]:
include_indices = np.array(include_indices)

In [24]:
input_data = input_data[include_indices]
output_data = output_data[include_indices]

In [25]:
random_ind = np.random.permutation(len(input_data))
random_ind.max(), random_ind.min(), random_ind

(2332714,
 0,
 array([  20724, 1105512, 2040733, ..., 1792859,  788273,  186204]))

In [26]:
input_data = input_data[random_ind]
output_data = output_data[random_ind]

In [27]:
input_data.shape, output_data.shape

((2332715, 68), (2332715,))

In [28]:
with open("../dataset/checkers_jump_input.pickle", "wb") as file:
    pickle.dump(input_data, file)
    
with open("../dataset/checkers_jump_output.pickle", "wb") as file:
    pickle.dump(output_data, file)

### preprocess data

In [16]:
with open("../dataset/checkers_white_input.pickle", "rb") as file:
    white_input_data = pickle.load(file)

In [20]:
len(white_input_data)

115831

In [21]:
white_input_data[0].reshape(-1, 8)

array([[0, 3, 0, 3, 0, 3, 0, 3],
       [3, 0, 3, 0, 3, 0, 3, 0],
       [0, 3, 0, 3, 0, 3, 0, 3],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0]])

In [44]:
def one_hot_encoding(input_data):
    new_input = []
    for inp in input_data:
        new_input.append([])
        for num in inp.flatten():
            base_4 = np.zeros(4)
            if num != 0:
                base_4[num-1] = 1
            new_input[-1].append(base_4)
    return new_input

In [45]:
bin_white_input = one_hot_encoding(white_input_data)

In [47]:
bin_white_input = np.array(bin_white_input).reshape(len(white_input_data), 12, 8, 4)

In [57]:
bin_white_input

array([[[[0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 1., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 1., 0.],
         [0., 0., 0., 0.],
         [0., 0., 1., 0.]],

        ...,

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [1., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
 

In [59]:
with open("../dataset/bin_white_input.pickle", "wb") as file:
    pickle.dump(bin_white_input, file)